<a href="https://colab.research.google.com/github/rmedi21/challenge-alura-store/blob/main/AluraStoreLatam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importación de datos



In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


#1. Análisis de facturación



In [ ]:
# formato
tiendas = [tienda, tienda2, tienda3, tienda4]

for i in range(len(tiendas)):
  tiendas[i].insert(0, "tienda", i + 1)

df_tiendas = pd.concat(tiendas)

df_tiendas['Fecha de Compra'] = pd.to_datetime(df_tiendas['Fecha de Compra'], format='%d/%m/%Y')


tienda                      int64
Producto                   object
Categoría del Producto     object
Precio                    float64
Costo de envío            float64
Fecha de Compra            object
Vendedor                   object
Lugar de Compra            object
Calificación                int64
Método de pago             object
Cantidad de cuotas          int64
lat                       float64
lon                       float64
dtype: object
tienda                             int64
Producto                          object
Categoría del Producto            object
Precio                           float64
Costo de envío                   float64
Fecha de Compra           datetime64[ns]
Vendedor                          object
Lugar de Compra                   object
Calificación                       int64
Método de pago                    object
Cantidad de cuotas                 int64
lat                              float64
lon                              float64
dtype: obje

In [ ]:
# facturacion por tienda
tiendas_group = df_tiendas.groupby("tienda")
tiendas_group.sum(numeric_only = True)

,Precio,Costo de envío,Calificación,Cantidad de cuotas,lat,lon
tienda,,,,,,
1,1.150880e+09,61377900.0,9381,6943,12656.70220,-176428.23565
2,1.116344e+09,59485100.0,9524,6847,12973.91840,-176318.64904
3,1.098020e+09,58516600.0,9550,7001,12826.06305,-176296.23942
4,1.038376e+09,55317400.0,9422,6829,12765.07747,-176318.49861


# 2. Ventas por categoría

# 3. Calificación promedio de la tienda


# 4. Productos más y menos vendidos

# 5. Envío promedio por tienda